**1. IMPORTER LES LIBRAIRIES**

In [7]:
import pandas as pd
import numpy as np

import re

# StopWord
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize 
from nltk.corpus import stopwords
from nltk.tokenize.regexp import RegexpTokenizer

# Normalisation lexicale
from nltk.stem.snowball import FrenchStemmer
import nltk
nltk.download('wordnet')
import spacy
#!python -m spacy download fr_core_news_md
nlp = spacy.load("fr_core_news_md")

# Ré-échantillonnage
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Vectorisation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Normalisation
from sklearn.preprocessing import StandardScaler

# Train_Test Split
from sklearn.model_selection import train_test_split

# Réduction de dimension
from sklearn.feature_selection import VarianceThreshold

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Huynh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Huynh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Huynh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**2. IMPORTER LE FICHIER**

In [8]:
df = pd.read_csv("2. Pre-processing & Feature engineering_1.csv")
# Remettre la variable 'date_experience' au format datetime
df['date_experience'] = pd.to_datetime(df['date_experience'])

df.info()
display(df.head())
print("Nombre de valeurs nulles :", df.isna().sum().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73846 entries, 0 to 73845
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   nom               73846 non-null  object        
 1   notes             73846 non-null  int64         
 2   pays              73846 non-null  object        
 3   nbr_avis          73846 non-null  int64         
 4   date_publication  73846 non-null  object        
 5   date_experience   73846 non-null  datetime64[ns]
 6   commentaire       73846 non-null  object        
 7   titre             73846 non-null  object        
 8   commentaires      73846 non-null  object        
 9   ponctuation       73846 non-null  int64         
 10  sentiment         73846 non-null  float64       
 11  sentiment_labels  73846 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 6.8+ MB


,nom,notes,pays,nbr_avis,date_publication,date_experience,commentaire,titre,commentaires,ponctuation,sentiment,sentiment_labels
0,GUY PEYRONNENC,5,FR,7,2 days ago,2024-04-09,commande simple et rapide,commandee simple et rapide,commandee simple et rapide commande simple et ...,0,0.225000,Positif
1,bernard auge,5,FR,1,4 days ago,2024-04-07,tres bon service,rapide et simple,rapide et simple tres bon service,0,0.483333,Positif
2,alain bruno,5,FR,3,"Mar 26, 2024",2024-03-20,No comment,Honorable bon suivi !,Honorable bon suivi !,1,0.687500,Positif
3,Martin Maurice,1,FR,41,2 days ago,2024-04-09,Cdiscount cautionne l'escroquerie en ce sens o...,Cdiscount cautionne l'escroquerie en ce…,Cdiscount cautionne l'escroquerie en ce… Cdisc...,1,0.450000,Positif
4,PHILIPPE PLENET,5,FR,13,7 hours ago,2024-04-11,"Simple, rapide, prix raisonnables et €commerce...",Simple,"Simple Simple, rapide, prix raisonnables et €c...",0,0.190000,Positif


Nombre de valeurs nulles : 0


**3. PRE-PROCESSING & FEATURE ENGINEERING_2**

*1) SUPPRESSION COMMENTAIRES > n MOTS*

In [9]:
# Supprimer les commentaires de plus de n mots
n = 100
df['nombre_element'] = df['commentaires'].apply(lambda x: len(x.split()))
df = df.drop(df.loc[(df['nombre_element'] > n)].index)

### Réinitialiser l'inde
df = df.reset_index(drop=True)

*2) SUPPRESSION DES VARIABLES INUTILES*

In [10]:
df = df[['notes', 'commentaires', 'ponctuation', 'sentiment']]

*3) STANDARDISATION DES VARIABLES 'ponctuation' ET 'sentiment'*

In [11]:
cols = ['ponctuation', 'sentiment']
sc = StandardScaler()
df[cols] = sc.fit_transform(df[cols])
df[cols] = sc.transform(df[cols])

# Changer le format : 'float64' >>> 'float32' pour contourner un possible problème de mémoire
df[cols]  = df[cols].astype('float32')

*4) STOP WORDS*

In [12]:
# 1. Mettre le texte en minuscule
df["commentaires"] = df["commentaires"].str.lower()

# 2. Tokenisation : Scinder les phrases en suite de mot
df["commentaires"] = df["commentaires"].apply(word_tokenize)

# 3. Appliquer le filtre de stopwords au corpus de texte
stop_words = set(stopwords.words('french'))
new_stop_words = [",", ".", "``", "@", "*", "(", ")", "...", "!", "?", "-", "_", ">", "<", ":", "/", "=", "--", "©", "~", ";", "\\", "\\\\", "qu'on", "m'a", "pri",  "ca", "j'ai", "disant","qu'ils", "qu'elles", "dit", "faire", "qu'il", "qu'elle","en", "EN", "En", ".", ",", "Je", "c'est", "s'est", 'a', 'à', 'â', 'abord', 'afin', 'ai', 'aie', 'ainsi', 'allaient', 'allons', 'après', 'assez', 'attendu', 'au', 'auquel', 'aura', 'auront', 'autre', 'autres', 'aux', 'auxquelles', 'auxquels', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avoir', 'ayant', 'b', 'bigre', 'boum', 'brrr', 'c', 'ça', 'car', 'ce', 'ceci', 'cela', 'celle', 'celle-ci', 'celle-là', 'celles', 'celles-ci', 'celles-là', 'celui', 'celui-ci', 'celui-là', 'cent', 'certain', 'certaine', 'certaines', 'certains', 'certes', 'ces', 'cet', 'cette', 'ceux', 'ceux-ci', 'ceux-là', 'chacun', 'chaque', 'chez', 'chiche','ci', 'cinq', 'cinquantaine', 'cinquante', 'cinquantième', 'cinquième', 'clac', 'clic', 'd', 'da', 'dans', 'de', 'debout', 'des', 'dès', 'desquelles', 'desquels', 'dessous', 'dessus', 'deux', 'deuxième', 'deuxièmement', 'devant', 'devers', 'devra', 'dire', 'dix', 'dix-huit', 'dixième', 'dix-neuf', 'dix-sept', 'douze', 'douzième', 'dring', 'du', 'duquel', 'e', 'elle', 'elle-même', 'elles', 'elles-mêmes', 'en', 'entre', 'envers', 'es', 'ès', 'est', 'et', 'etant', 'étaient', 'étais', 'était', 'étant', 'etc', 'été', 'etre', 'être', 'eux', 'eux-mêmes', 'f', 'fais', 'faisaient', 'faisant', 'fait', 'feront', 'fi', 'flac', 'floc', 'font', 'g', 'gens', 'h', 'huit', 'huitième', 'hum', 'i', 'il', 'ils', 'j', 'je', 'k', 'l', 'la', 'là', 'laquelle', 'las', 'le', 'lequel', 'les', 'lès', 'lesquelles', 'lesquels', 'leur', 'leurs', 'lorsque', 'lui', 'lui-même', 'm', 'ma', 'maint', 'mais', 'me', 'même', 'mêmes', 'mes', 'mien', 'mienne', 'miennes', 'miens', 'mille', 'moi', 'moi-même', 'mon', 'neuf', 'neuvième', 'nos', 'notre', 'nôtre', 'nôtres', 'nous', 'nous-mêmes', 'on', 'ont', 'onze', 'onzième', 'ore', 'ou', 'où', 'outre', 'p', 'paf', 'pan', 'par', 'partant', 'peut', 'peuvent', 'peux','premier', 'première', 'premièrement', 'puisque', 'q', 'qu', 'quanta', 'quant-à-soi', 'quarante', 'quatorze', 'quatre', 'quatre-vingt', 'quatrième', 'quatrièmement', 'que', 'quel', 'quelle', 'quelles', 'quelque', 'quelques', "quelqu'un", 'quels', 'qui', 'quiconque', 'quinze', 'quoi', 'quoique', 'r', 'revoici', 'revoilà', 's', 'sa', 'sacrebleu', 'sapristi', 'se', 'seize', 'selon', 'sept', 'septième', 'sera', 'seront', 'ses', 'si', 'sien', 'sienne', 'siennes', 'siens', 'sinon', 'six', 'sixième', 'soi', 'soi-même', 'soit', 'soixante', 'son', 'sont', 'sous', 'suis', 't', 'ta', 'tac', 'te', 'té', 'tel', 'telle', 'telles', 'tels', 'tenant', 'tes', 'tic', 'tien', 'tienne', 'tiennes', 'tiens', 'toc', 'toi', 'toi-même', 'ton', 'treize', 'trente', 'très', 'trois', 'troisième', 'troisièmement', 'tsoin', 'tsouin', 'tu', 'u', 'un', 'une', 'unes', 'uns', 'v', 'va', 'vais', 'vas', 'vé', 'vingt', 'vivat', 'vlan', 'vont', 'vos', 'votre', 'vôtre', 'vôtres', 'vous', 'vous-mêmes', 'vu', 'w', 'x', 'y', 'z', 'devrait', 'dos', 'droite', 'faites', 'haut', 'mine', 'mot', 'nommés', 'parce','voient', 'état', 'étions']
# Ajouter à la liste des stopwords, des éléments de syntaxe que nous avons repéré et qui ne servent pas à l'analyse du texte
stop_words.update(new_stop_words)
# Définir la fonction stop_words_filtering
def stop_words_filtering(mots) : 
    tokens = []
    for mot in mots:
        if mot not in stop_words:
            tokens.append(mot)
    return tokens
# Appliquer la fonction à la colonne 'commentaires'
df["commentaires"] = df["commentaires"].apply(stop_words_filtering)

# 4. Reconstitution des phrases après la tokenization
for i in range (0, len(df['commentaires'])):
    df.loc[i, 'commentaires'] = ' '.join(df.loc[i, 'commentaires'])

# 5. Supprimer les regexs identifiés
# Regex pour tous les '.', '..', '...', ...
for i in range (0, len(df['commentaires'])):
    df.loc[i, 'commentaires'] = re.sub(r"\.+", '', df.loc[i, 'commentaires'])
# Regex pour tous les chiffres et nombres
for i in range (0, len(df['commentaires'])):
    df.loc[i, 'commentaires'] = re.sub(r"[0-9]+", '', df.loc[i, 'commentaires'])

*5) NORMALISATION LEXICALE*

In [13]:
df.head()

,notes,commentaires,ponctuation,sentiment
0,5,commandee simple rapide commande simple rapide,-0.148002,-1.112980
1,5,rapide simple tres bon service,-0.148002,1.881152
2,5,honorable bon suivi,-0.090812,4.247482
3,1,cdiscount cautionne l'escroquerie ce… cdiscoun...,-0.090812,1.494813
4,5,simple simple rapide prix raisonnables €commer...,-0.148002,-1.518636


*a. Lemmatisation (en français)*

In [14]:
df_lem = df

In [15]:
def french_lem(commentaire) :
    sortie = []
    commentaire = nlp(str(commentaire))
    for mot in commentaire :
        radical = mot.lemma_
        if (radical not in sortie) : sortie.append(radical)
    resultat = ' '.join(sortie)
    return resultat
    
df_lem['commentaires'] = df_lem['commentaires'].apply(lambda x : french_lem(x))

In [16]:
# Vérification
df_lem.head()

,notes,commentaires,ponctuation,sentiment
0,5,commandee simple rapide commande,-0.148002,-1.112980
1,5,rapide simple tres bon service,-0.148002,1.881152
2,5,honorable bon suivre,-0.090812,4.247482
3,1,cdiscount cautionner le escroquerie ce … sens ...,-0.090812,1.494813
4,5,simple rapide prix raisonnable euro commerce f...,-0.148002,-1.518636


*b. Stemming/Racinisation (en français)*

In [17]:
df_stem = df

In [18]:
stemmer = FrenchStemmer()
def stemming(commentaire) :
    liste_mots = commentaire.split()
    sortie = []
    for mot in liste_mots :
        radical = stemmer.stem(mot)
        if (radical not in sortie) : sortie.append(radical)
    resultat = ' '.join(sortie)
    return resultat

df_stem['commentaires'] = df_stem['commentaires'].apply(lambda x : stemming(x))

In [19]:
# Vérification
df_stem.head()

,notes,commentaires,ponctuation,sentiment
0,5,commande simpl rapid command,-0.148002,-1.112980
1,5,rapid simpl tre bon servic,-0.148002,1.881152
2,5,honor bon suivr,-0.090812,4.247482
3,1,cdiscount caution le escroquer ce … sen achet ...,-0.090812,1.494813
4,5,simpl rapid prix raison euro commerc franc,-0.148002,-1.518636


*6) VECTORISATION : TF-IDF*

In [43]:
vec_tfidf = TfidfVectorizer()

*a. Pour la Lemmatisation*

In [44]:
# Séparation Target/Features
X_lem = df_lem.drop('notes', axis = 1)
y_lem = df_lem['notes']

# Train_Test_Split
X_train_lem, X_test_lem, y_train_lem, y_test_lem = train_test_split(X_lem, y_lem, test_size=0.2, random_state = 42)

In [45]:
X_train = vec_tfidf.fit_transform(X_train_lem['commentaires'])
X_test = vec_tfidf.transform(X_test_lem['commentaires'])

In [48]:
X_train = pd.DataFrame(X_train.astype('float32').toarray(), index = X_train_lem.index)
X_train[cols] = X_train_lem[cols]
X_train_lem = X_train

In [50]:
X_test = pd.DataFrame(X_test.astype('float32').toarray(), index = X_test_lem.index)
X_test[cols] = X_test_lem[cols]
X_test_lem = X_test

In [51]:
# Vérification shape
print ("X_train_lem =", X_train_lem.shape)
print ("y_train_lem =", y_train_lem.shape)
print ("X_test_lem =", X_test_lem.shape)
print ("y_test_lem =", y_test_lem.shape)

X_train_lem = (55394, 17828)
y_train_lem = (55394,)
X_test_lem = (13849, 17828)
y_test_lem = (13849,)


*b. Pour le Stemming*

In [52]:
# Séparation Target/Features
X_stem = df_stem.drop('notes', axis = 1)
y_stem = df_stem['notes']

# Train_Test_Split
X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split(X_stem, y_stem, test_size=0.2, random_state = 42)

In [53]:
X_train = vec_tfidf.fit_transform(X_train_stem['commentaires'])
X_test = vec_tfidf.transform(X_test_stem['commentaires'])

In [54]:
X_train = pd.DataFrame(X_train.astype('float32').toarray(), index = X_train_stem.index)
X_train[cols] = X_train_stem[cols]
X_train_stem = X_train

In [55]:
X_test = pd.DataFrame(X_test.astype('float32').toarray(), index = X_test_stem.index)
X_test[cols] = X_test_stem[cols]
X_test_stem = X_test

In [56]:
# Vérification shape
print ("X_train_stem =", X_train_stem.shape)
print ("y_train_stem =", y_train_stem.shape)
print ("X_test_stem =", X_test_stem.shape)
print ("y_test_stem =", y_test_stem.shape)

X_train_stem = (55394, 17828)
y_train_stem = (55394,)
X_test_stem = (13849, 17828)
y_test_stem = (13849,)


*7) REDUCTION DE DIMENSION*

*a. Pour la Lemmatisation*

In [59]:
# Supprimer les variables de variance nulle
sel = VarianceThreshold(threshold = 1e-4)
X_train_lem_rd = sel.fit_transform(X_train_lem)
X_test_lem_rd = sel.transform(X_test_lem)

In [60]:
# Vérification shape
print ("X_train_lem_rd =", X_train_lem_rd.shape)
print ("y_train_lem =", y_train_lem.shape)
print ("X_test_lem_rd =", X_test_lem_rd.shape)
print ("y_test_lem =", y_test_lem.shape)

X_train_lem_rd = (55394, 1101)
y_train_lem = (55394,)
X_test_lem_rd = (13849, 1101)
y_test_lem = (13849,)


In [63]:
X_train_lem_rd = pd.DataFrame(X_train_lem_rd, index = X_train_lem.index)
X_test_lem_rd = pd.DataFrame(X_test_lem_rd, index = X_test_lem.index)

*b. Pour le Stemming*

In [64]:
# Supprimer les variables de variance nulle
sel1 = VarianceThreshold(threshold = 1e-4)
X_train_stem_rd = sel1.fit_transform(X_train_stem)
X_test_stem_rd = sel1.transform(X_test_stem)

In [65]:
# Vérification shape
print ("X_train_stem_rd =", X_train_stem_rd.shape)
print ("y_train_stem =", y_train_stem.shape)
print ("X_test_stem_rd =", X_test_stem_rd.shape)
print ("y_test_stem =", y_test_stem.shape)

X_train_stem_rd = (55394, 1101)
y_train_stem = (55394,)
X_test_stem_rd = (13849, 1101)
y_test_stem = (13849,)


In [66]:
X_train_stem_rd = pd.DataFrame(X_train_stem_rd, index = X_train_stem.index)
X_test_stem_rd = pd.DataFrame(X_test_stem_rd, index = X_test_stem.index)

**4. DATAFRAME DANS UN FICHIER .CSV**

*1) Lemmatisation*

In [68]:
# a. Lemmatisation (Sans réduction de dimension)
# new_df_lem = pd.concat((X_train_lem, X_test_lem))
# new_df_lem['y'] = pd.concat((y_train_lem, y_test_lem))
# new_df_lem.to_csv("4. Lemmatisation_TF-IDF(Sans_Reduction).csv", index=False)

### Nous obtenons des fichiers de plusieurs Giga bien trop compliqués à traiter.

In [72]:
# b. Lemmatisation (Avec Réduction de dimension)
new_df_lem_rd = pd.concat((X_train_lem_rd, X_test_lem_rd))
new_df_lem_rd['y'] = pd.concat((y_train_lem, y_test_lem))
new_df_lem_rd.to_csv("4. Lemmatisation_TF-IDF(Avec_Reduction).csv", index=False)

*2) Stemming*

In [73]:
# a. Stemming (Sans réduction de dimension)
# new_df_stem = pd.concat((X_train_stem, X_test_stem))
# new_df_stem['y'] = pd.concat((y_train_stem, y_test_stem))
# new_df_stem.to_csv("4. Stemming_TF-IDF(Sans_Reduction).csv", index=False)

### Nous obtenons des fichiers de plusieurs Giga bien trop compliqués à traiter.

In [74]:
# b. Stemming (Avec Réduction de dimension)
new_df_stem_rd = pd.concat((X_train_stem_rd, X_test_stem_rd))
new_df_stem_rd['y'] = pd.concat((y_train_stem, y_test_stem))
new_df_stem_rd.to_csv("4. Stemming_TF-IDF(Avec_Reduction).csv", index=False)